# Pràctica 3 - PLH

### Realitzada pels alumnes Lluc Furriols i Pau Prat Moreno

In [1]:
'''
import os 
f = open("/dev/null", "w")
os.dup2(f.fileno(), 2)
f.close()

import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()
'''

'\nimport os \nf = open("/dev/null", "w")\nos.dup2(f.fileno(), 2)\nf.close()\n\nimport nltk\nimport ssl\n\ntry:\n    _create_unverified_https_context = ssl._create_unverified_context\nexcept AttributeError:\n    pass\nelse:\n    ssl._create_default_https_context = _create_unverified_https_context\n\nnltk.download()\n'

## Importació de llibreries

In [26]:
import nltk
nltk.download('punkt', quiet=True) # Tokenitzador
nltk.download('averaged_perceptron_tagger', quiet=True) # Etiquetador POS
nltk.download('maxent_ne_chunker', quiet=True) # Etiquetador Entitats Anomenades
nltk.download('words', quiet=True)

True

## Carreguem les dades

In [27]:
nltk.download('conll2002')
from nltk.corpus import conll2002

train_esp = conll2002.iob_sents('esp.train') # Train, 
val_esp = conll2002.iob_sents('esp.testa') # Val
test_esp = conll2002.iob_sents('esp.testb') # Test

train_ned = conll2002.iob_sents('ned.train') # Train
val_ned = conll2002.iob_sents('ned.testa') # Val
test_ned = conll2002.iob_sents('ned.testb') # Test

[nltk_data] Downloading package conll2002 to /Users/pau/nltk_data...
[nltk_data]   Package conll2002 is already up-to-date!


In [50]:
train_esp

[[('Melbourne', 'NP', 'B-LOC'), ('(', 'Fpa', 'O'), ('Australia', 'NP', 'B-LOC'), (')', 'Fpt', 'O'), (',', 'Fc', 'O'), ('25', 'Z', 'O'), ('may', 'NC', 'O'), ('(', 'Fpa', 'O'), ('EFE', 'NC', 'B-ORG'), (')', 'Fpt', 'O'), ('.', 'Fp', 'O')], [('-', 'Fg', 'O')], ...]

In [51]:
def get_token(sequence):
    """
    Retorna una llista de tokens.
    """
    return [[(token) for token, pos, entity in sentence] for sentence in sequence]

def get_token_entity(sequence):
    """
    Retorna una llista de tokens i les seves entitats.
    """
    return [[(token, entity) for token, pos, entity in sentence] for sentence in sequence]

In [52]:
print(get_token_entity(train_esp)[0])

[('Melbourne', 'B-LOC'), ('(', 'O'), ('Australia', 'B-LOC'), (')', 'O'), (',', 'O'), ('25', 'O'), ('may', 'O'), ('(', 'O'), ('EFE', 'B-ORG'), (')', 'O'), ('.', 'O')]


# First execution with no modifications

In [30]:
from nltk.tag import CRFTagger
import pycrfsuite

ct = CRFTagger(feature_func=None)
#train and test sets without the postag
train_esp_bio = get_token_entity(train_esp)
test_esp_bio = get_token_entity(test_esp)
print("How train data looks like: ",train_esp_bio[0])

ct.train(train_esp_bio, 'model.crf.tagger')
print(ct.tag_sents([[token for token, _ in test_esp_bio[0]]])[0])

[('Melbourne', 'B-LOC'), ('(', 'O'), ('Australia', 'B-LOC'), (')', 'O'), (',', 'O'), ('25', 'O'), ('may', 'O'), ('(', 'O'), ('EFE', 'B-ORG'), (')', 'O'), ('.', 'O')]
[('La', 'B-LOC'), ('Coruña', 'I-LOC'), (',', 'O'), ('23', 'O'), ('may', 'O'), ('(', 'O'), ('EFECOM', 'B-ORG'), (')', 'O'), ('.', 'O')]


###  Probar el model en el conjunt de test

In [53]:
y_pred = ct.tag_sents([[token for token, _ in sent] for sent in test_esp_bio])


[('La', 'B-LOC'), ('Coruña', 'I-LOC'), (',', 'O'), ('23', 'O'), ('may', 'O'), ('(', 'O'), ('EFECOM', 'B-ORG'), (')', 'O'), ('.', 'O')]


In [48]:
def extract_entities(tagged_words):
    entities = []
    current_entity = []
    current_type = None
    current_start_index = None  # Afegim una variable per guardar l'índex d'inici

    for index, (word, tag) in enumerate(tagged_words):
        if tag.startswith('B-'):  # Començament d'una nova entitat
            if current_entity:  # Si hi havia una entitat en construcció, l'afegim abans de començar la nova
                entities.append((current_start_index, index - 1, current_type))
            current_entity = [word]  # Comencem una nova entitat
            current_start_index = index  # Guardem l'índex d'inici de l'entitat actual
            current_type = tag[2:]  # Guardem el tipus d'entitat sense el prefix B-
        elif tag.startswith('I-') and current_type == tag[2:]:  # Continuació de la mateixa entitat
            current_entity.append(word)
        else:  # Si no és una continuació de la mateixa entitat o és 'O'
            if current_entity:  # Finalitzem l'entitat actual si n'hi ha una
                entities.append((current_start_index, index - 1, current_type))
                current_entity = []
                current_type = None
            if tag == 'O':
                continue
            else:  # Codificació IO o canvi d'entitat amb I-
                current_entity = [word]
                current_start_index = index
                current_type = tag[2:]  # Possible en cas de codificació IO

    # Assegurar-se d'afegir l'última entitat si la llista no acaba en 'O'
    if current_entity:
        entities.append((current_start_index, index, current_type))

    return entities


def evaluate_entities(y_test, y_pred, print_errors=False):
    """
    Evalua el model. Rep les llistes de les entitats reals i predites.
    Els elements de les llistes són tuples (token, entitat).
    Si print_errors = True, imprimeix les entitats mal classificades.
    Retorna la precisió, recall i F1 score.
    """
    total_entities = 0
    correct_entities = 0

    for sent_test, sent_pred in zip(y_test, y_pred):
        true_entities = extract_entities(sent_test)
        pred_entities = extract_entities(sent_pred)
        
        # Comptar entitats reals
        entities_sentence = len(true_entities)
        # Entitats correctament predites
        entities_predicted = len([e for e in true_entities if e in pred_entities])

        # Portem el compte de totes les entitats
        total_entities += entities_sentence
        # Portem el compte de les entitats correctament predites
        correct_entities += entities_predicted

        if print_errors:
            if entities_sentence != entities_predicted:
                print('Real sentence:', sent_test)
                print('Predicted:', sent_pred)
                print('Difference in:', (set(true_entities) - set(pred_entities)))
                print()

        
    if total_entities == 0:
        return 0, 0, 0, 0

    precision = correct_entities / total_entities
    recall = correct_entities / total_entities
    f1_score = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0

    return precision, recall, f1_score


precision, recall, f1_score = evaluate_entities(test_esp_bio, y_pred, print_errors=True)
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1_score:.2f}')


Real sentence: [('Las', 'O'), ('reservas', 'O'), ('"', 'O'), ('on', 'O'), ('line', 'O'), ('"', 'O'), ('de', 'O'), ('billetes', 'O'), ('aéreos', 'O'), ('a', 'O'), ('través', 'O'), ('de', 'O'), ('Internet', 'B-MISC'), ('aumentaron', 'O'), ('en', 'O'), ('España', 'B-LOC'), ('un', 'O'), ('300', 'O'), ('por', 'O'), ('ciento', 'O'), ('en', 'O'), ('el', 'O'), ('primer', 'O'), ('trimestre', 'O'), ('de', 'O'), ('este', 'O'), ('año', 'O'), ('con', 'O'), ('respecto', 'O'), ('al', 'O'), ('mismo', 'O'), ('período', 'O'), ('de', 'O'), ('1999', 'O'), (',', 'O'), ('aseguró', 'O'), ('hoy', 'O'), ('Iñigo', 'B-PER'), ('García', 'I-PER'), ('Aranda', 'I-PER'), (',', 'O'), ('responsable', 'O'), ('de', 'O'), ('comunicación', 'O'), ('de', 'O'), ('Savia', 'B-ORG'), ('Amadeus', 'I-ORG'), ('.', 'O')]
Predicted: [('Las', 'O'), ('reservas', 'O'), ('"', 'O'), ('on', 'O'), ('line', 'O'), ('"', 'O'), ('de', 'O'), ('billetes', 'O'), ('aéreos', 'O'), ('a', 'O'), ('través', 'O'), ('de', 'O'), ('Internet', 'B-MISC'), ('a

## Canviar feature functions i codificacio

In [7]:
import string

class FeatureGetter:
    """
    Aquesta classe s'utilitza per obtenir diferents característiques d'un token de text.
    """
    def __init__(self):
        pass

    def has_capitalization(self, token):
        """
        Comprova si el token conté alguna lletra majúscula.
        """
        return any(char.isupper() for char in token)

    def has_digit(self, token):
        """
        Comprova si el token conté algun dígit.
        """
        return any(char.isdigit() for char in token)

    def has_punctuation(self, token):
        """
        Comprova si el token conté algun signe de puntuació.
        """
        return any(char in string.punctuation for char in token)

    def get_prefix(self, token, n=3):
        """
        Obté el prefix del token amb una longitud de n caràcters.
        """
        return token[:n] if len(token) > n else token

    def get_suffix(self, token, n=3):
        """
        Obté el sufix del token amb una longitud de n caràcters.
        """
        return token[-n:] if len(token) > n else token

    def all_caps(self, token):
        """
        Comprova si totes les lletres del token són majúscules.
        """
        return token.isupper()

    def is_capitalized(self, token):
        """
        Comprova si la primera lletra del token és majúscula.
        """
        return token[0].isupper()

    def get_features(self, token, add_prefix_suffix=True):
        """
        Obté un diccionari amb les característiques del token.
        """
        features = {
            'bias': 1.0,
            'has_capitalization': self.has_capitalization(token),
            'has_digit': self.has_digit(token),
            'has_punctuation': self.has_punctuation(token),
            'all_caps': self.all_caps(token),
            'is_capitalized': self.is_capitalized(token),
        }

        if add_prefix_suffix:
            features['prefix'] = self.get_prefix(token)
            features['suffix'] = self.get_suffix(token)

        return features

# Exemple d'ús:
feature_getter = FeatureGetter()
token_features = feature_getter.get_features('Barcelona')
print(token_features)

{'bias': 1.0, 'has_capitalization': True, 'has_digit': False, 'has_punctuation': False, 'all_caps': False, 'is_capitalized': True, 'prefix': 'Bar', 'suffix': 'ona'}


## Codificació BIO

In [36]:
def extract_entities(tagged_words, encoding='BIO'):
    """
    Extreu les entitats d'una llista de paraules etiquetades segons l'encoding especificat.
    
    Arguments:
        tagged_words: una llista de tuples (word, tag), on 'word' és una paraula del text i 'tag' és la seva etiqueta (BIO/BIOE/BIOW/IO).
        encoding: el tipus de codificació utilitzat per les etiquetes ('BIO', 'BIOW', 'BIOE', 'IO').
        
    Retorna:
        Una llista de tuples (start_index, end_index, entity_type) que representen les entitats trobades.
        'start_index' i 'end_index' són els índexs on comença i acaba l'entitat en la llista de paraules, i 'entity_type' és el tipus d'entitat.
    """

    entities = []  # Llista on guardarem les entitats trobades
    current_entity = []  # Guarda les paraules de l'entitat actual
    current_type = None  # Tipus de l'entitat actual
    current_start_index = None  # Índex d'inici de l'entitat actual

    for index, (word, tag) in enumerate(tagged_words):
        tag_type = None if tag == 'O' else tag[2:]

        if tag == 'O':
            if current_entity:
                entities.append((current_start_index, index - 1, current_type))
                current_entity = []
                current_type = None
            continue

        if encoding == 'IO':
            if tag_type != current_type:
                if current_entity:
                    entities.append((current_start_index, index - 1, current_type))
                current_entity = [word]
                current_start_index = index
                current_type = tag_type
            else:
                current_entity.append(word)
        else:
            tag_prefix = tag[:1]
            if tag_prefix in ['B', 'W']:  # Començament d'una nova entitat o entitat de paraula única
                if current_entity:
                    entities.append((current_start_index, index - 1, current_type))
                current_entity = [word]
                current_start_index = index
                current_type = tag_type
                if tag_prefix == 'W':  # Si és una entitat de paraula única, la tanquem immediatament
                    entities.append((current_start_index, index, current_type))
                    current_entity = []
                    current_type = None
            elif tag_prefix == 'I' and current_type == tag_type:
                current_entity.append(word)
            elif encoding == 'BIOE' and tag_prefix == 'E' and current_type == tag_type:
                current_entity.append(word)
                entities.append((current_start_index, index, current_type))
                current_entity = []
                current_type = None
            else:
                if current_entity:
                    entities.append((current_start_index, index - 1, current_type))
                    current_entity = []
                current_type = None

    if current_entity:
        entities.append((current_start_index, index, current_type))

    return entities

tagged_words = get_token_entity(train_esp)[3]

entities = extract_entities(tagged_words, "BIO")
print(entities)


[(1, 4, 'PER'), (6, 7, 'PER'), (38, 38, 'MISC')]
[('El', 'O'), ('Abogado', 'B-PER'), ('General', 'I-PER'), ('del', 'I-PER'), ('Estado', 'I-PER'), (',', 'O'), ('Daryl', 'B-PER'), ('Williams', 'I-PER'), (',', 'O'), ('subrayó', 'O'), ('hoy', 'O'), ('la', 'O'), ('necesidad', 'O'), ('de', 'O'), ('tomar', 'O'), ('medidas', 'O'), ('para', 'O'), ('proteger', 'O'), ('al', 'O'), ('sistema', 'O'), ('judicial', 'O'), ('australiano', 'O'), ('frente', 'O'), ('a', 'O'), ('una', 'O'), ('página', 'O'), ('de', 'O'), ('internet', 'O'), ('que', 'O'), ('imposibilita', 'O'), ('el', 'O'), ('cumplimiento', 'O'), ('de', 'O'), ('los', 'O'), ('principios', 'O'), ('básicos', 'O'), ('de', 'O'), ('la', 'O'), ('Ley', 'B-MISC'), ('.', 'O')]


In [55]:
get_token_entity(train_esp)[3]

[('La', 'O'),
 ('petición', 'O'),
 ('del', 'O'),
 ('Abogado', 'B-PER'),
 ('General', 'I-PER'),
 ('tiene', 'O'),
 ('lugar', 'O'),
 ('después', 'O'),
 ('de', 'O'),
 ('que', 'O'),
 ('un', 'O'),
 ('juez', 'O'),
 ('del', 'O'),
 ('Tribunal', 'B-ORG'),
 ('Supremo', 'I-ORG'),
 ('del', 'O'),
 ('estado', 'O'),
 ('de', 'O'),
 ('Victoria', 'B-LOC'),
 ('(', 'O'),
 ('Australia', 'B-LOC'),
 (')', 'O'),
 ('se', 'O'),
 ('viera', 'O'),
 ('forzado', 'O'),
 ('a', 'O'),
 ('disolver', 'O'),
 ('un', 'O'),
 ('jurado', 'O'),
 ('popular', 'O'),
 ('y', 'O'),
 ('suspender', 'O'),
 ('el', 'O'),
 ('proceso', 'O'),
 ('ante', 'O'),
 ('el', 'O'),
 ('argumento', 'O'),
 ('de', 'O'),
 ('la', 'O'),
 ('defensa', 'O'),
 ('de', 'O'),
 ('que', 'O'),
 ('las', 'O'),
 ('personas', 'O'),
 ('que', 'O'),
 ('lo', 'O'),
 ('componían', 'O'),
 ('podían', 'O'),
 ('haber', 'O'),
 ('obtenido', 'O'),
 ('información', 'O'),
 ('sobre', 'O'),
 ('el', 'O'),
 ('acusado', 'O'),
 ('a', 'O'),
 ('través', 'O'),
 ('de', 'O'),
 ('la', 'O'),
 ('página

## Codificació IO

In [65]:
def bio_to_io(bio_tagged_sentences):
    """
    Converteix les etiquetes de múltiples frases des de la codificació BIO a IO.
    
    Arguments:
        bio_tagged_sentences: Una llista de llistes de tuples (word, tag) on 'tag' és en codificació BIO.
    
    Retorna:
        Una llista de llistes de tuples (word, io_tag) on 'io_tag' és en codificació IO per cada frase.
    """
    io_tagged_sentences = []
    
    for sentence in bio_tagged_sentences:
        io_tagged_sentence = []
        for word, tag in sentence:
            if tag.startswith('B-'):
                # Canvia B- per I-
                io_tagged_sentence.append((word, 'I-' + tag[2:]))
            elif tag.startswith('I-'):
                io_tagged_sentence.append((word, tag))
            else:
                # Manté les etiquetes 'O' tal com estan
                io_tagged_sentence.append((word, 'O'))
        io_tagged_sentences.append(io_tagged_sentence)
    
    return io_tagged_sentences


tagged_words = get_token_entity(train_esp)
tagged_words_io = bio_to_io(tagged_words)
print(tagged_words[3])
print(tagged_words_io[3])

[('La', 'O'), ('petición', 'O'), ('del', 'O'), ('Abogado', 'B-PER'), ('General', 'I-PER'), ('tiene', 'O'), ('lugar', 'O'), ('después', 'O'), ('de', 'O'), ('que', 'O'), ('un', 'O'), ('juez', 'O'), ('del', 'O'), ('Tribunal', 'B-ORG'), ('Supremo', 'I-ORG'), ('del', 'O'), ('estado', 'O'), ('de', 'O'), ('Victoria', 'B-LOC'), ('(', 'O'), ('Australia', 'B-LOC'), (')', 'O'), ('se', 'O'), ('viera', 'O'), ('forzado', 'O'), ('a', 'O'), ('disolver', 'O'), ('un', 'O'), ('jurado', 'O'), ('popular', 'O'), ('y', 'O'), ('suspender', 'O'), ('el', 'O'), ('proceso', 'O'), ('ante', 'O'), ('el', 'O'), ('argumento', 'O'), ('de', 'O'), ('la', 'O'), ('defensa', 'O'), ('de', 'O'), ('que', 'O'), ('las', 'O'), ('personas', 'O'), ('que', 'O'), ('lo', 'O'), ('componían', 'O'), ('podían', 'O'), ('haber', 'O'), ('obtenido', 'O'), ('información', 'O'), ('sobre', 'O'), ('el', 'O'), ('acusado', 'O'), ('a', 'O'), ('través', 'O'), ('de', 'O'), ('la', 'O'), ('página', 'O'), ('CrimeNet', 'B-MISC'), ('.', 'O')]
[('La', 'O'),

In [ ]:
'''from nltk.tag import CRFTagger
import pycrfsuite

ct = CRFTagger(feature_func=None)
#train and test sets without the postag
train_esp_bio = get_token_entity(train_esp)
test_esp_bio = get_token_entity(test_esp)
print(train_esp_bio[0])

#y_test is the true labels
y_test = [[iob for word, iob in sent] for sent in test_esp_bio]
print(y_test[0])

#train the model
ct.train(train_esp_bio, 'model.crf.tagger')

#predict the labels
y_pred = ct.tag_sents([[word for word, iob in sent] for sent in test_esp_bio])
y_pred = [[iob for word, iob in sent] for sent in y_pred]
print(y_pred[0])


#show the confusion matrix
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Flatten y_test and y_pred
y_test_flat = [iob for sent in y_test for iob in sent]
y_pred_flat = [iob for sent in y_pred for iob in sent]

# Generate confusion matrix
cm = confusion_matrix(y_test_flat, y_pred_flat)

# Visualize confusion matrix
plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.show()

#show the classification report
from sklearn.metrics import classification_report
print(classification_report(y_test_flat, y_pred_flat))
ct.accuracy(test_esp_bio)'''

In [18]:
def evaluation(true_entities, pred_entities):
    """
    Avalua la predicció de les entitats amb precisió, record i F1-score.
    """
    true_positives = len(set(true_entities) & set(pred_entities))
    if true_positives == 0:
        return 0, 0, 0
    precision = true_positives / len(pred_entities)
    recall = true_positives / len(true_entities)
    f1_score = 2 * precision * recall / (precision + recall)
    return precision, recall, f1_score

In [ ]:
from nltk.tag import CRFTagger
import pycrfsuite

ct = CRFTagger(feature_func=feature_getter.get_features)

train_esp_bio = get_token_entity(train_esp)
test_esp_bio = get_token_entity(test_esp)

ct.train(train_esp_bio, 'model.crf.tagger')

# Test the model
test_pred = ct.tag_sents(get_token(test_esp))

